https://adventofcode.com/2023/day/25

In [1]:
from itertools import combinations, pairwise
from functools import lru_cache
import random
from collections import Counter

import networkx as nx

In [2]:
with open("data/25.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr
"""

In [4]:
def parse_puzzle(puzzle):
    L = []
    for line in puzzle.splitlines():
        node, others = line.split(": ")
        L.append((node, others.split()))
    return L


def load_graph(puzzle):
    G = nx.Graph()
    for node, others in parse_puzzle(puzzle):
        for other in others:
            G.add_edge(node, other)
    return G

In [5]:
G = load_graph(data)

try:
    del d1
except NameError:
    pass


@lru_cache(maxsize=None)
def d1(node):
    return nx.descendants_at_distance(G, node, 1)

In [6]:
nodes = list(G)
len(nodes)

1440

In [7]:
cut_candidates = {
    frozenset((a, b)) for (a, b) in combinations(G, 2) if b in d1(a) and not (d1(a) & d1(b))
}
len(cut_candidates)

3179

In [8]:
random.choices(nodes, k=2)

['xsf', 'pdh']

In [9]:
nx.shortest_path(G, *random.choices(nodes, k=2))

['pgk',
 'nxm',
 'mtx',
 'xrc',
 'clb',
 'brd',
 'hnz',
 'ddc',
 'kjp',
 'fkt',
 'kgc',
 'mvq']

In [10]:
c = Counter()
for _ in range(1000):
    for edge in pairwise(nx.shortest_path(G, *random.choices(nodes, k=2))):
        edgefs = frozenset(edge)
        if edgefs in cut_candidates:
            c[edgefs] += 1

In [11]:
c.most_common(10)

[(frozenset({'brd', 'clb'}), 267),
 (frozenset({'bbz', 'jxd'}), 132),
 (frozenset({'glz', 'mxd'}), 120),
 (frozenset({'clb', 'cqz'}), 78),
 (frozenset({'clb', 'pfz'}), 69),
 (frozenset({'brd', 'qqb'}), 66),
 (frozenset({'jxd', 'vdr'}), 65),
 (frozenset({'bcr', 'brd'}), 60),
 (frozenset({'clb', 'lqg'}), 54),
 (frozenset({'kdm', 'mxd'}), 54)]

In [12]:
[tuple(edgefs) for (edgefs, count) in c.most_common(3)]

[('clb', 'brd'), ('bbz', 'jxd'), ('glz', 'mxd')]

In [13]:
G1 = G.copy()
G1.remove_edges_from(tuple(edgefs) for (edgefs, count) in c.most_common(3))
cc = list(nx.connected_components(G1))
if len(cc) == 2:
    print(len(cc[0]) * len(cc[1]))

518391


This works because the groups are of roughly equal size, and cut candidate edges are fairly evenly distributed.